## Unzip model files and install dependencies

In [ ]:
!unzip /content/DetectGPT.zip
!pip install -r /content/DetectGPT/requirements.txt
!pip install sentencepiece
!pip install transformers
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip uninstall -y Pillow
!pip install Pillow

In [ ]:
# Empties CUDA memory cache - to avoid CUDA out of memory error
import torch
torch.cuda.empty_cache()

In [ ]:
cd DetectGPT

## Mount Google Drive to save outputs to

In [ ]:
from google.colab import drive
import shutil
drive.mount('/content/drive')

In [ ]:
# Run DetectGPT with user input
# !python local_infer.py

## AIGC Detector execution

In [ ]:
from model import GPT2PPLV2 as GPT2PPL
import csv
import sys
import torch
import shutil

# input_file = '/content/DetectGPT/data/variant_1_full.csv'     # Replace input file path to file containing variant content
# output_file = '/content/DetectGPT/data/output_variant_1.csv'  # Replace output file path to file containing DetectGPT outputs

# Initialize the model
model = GPT2PPL()

# Open input and output files
with open(input_file, newline='', encoding="utf-8") as infile, open(output_file, 'w', newline='', encoding="utf-8") as outfile:

    # Create reader and writer objects
    reader_obj = csv.reader(infile)
    writer_obj = csv.writer(outfile)

    # Read and write the header, and add a new column for results
    header = next(reader_obj)
    header.append("DetectGPT_answer_human_full")
    header.append("DetectGPT_answer_GPT_full")
    writer_obj.writerow(header)

    # Iterate over each row in the csv file using reader object
    for idx, row in enumerate(reader_obj):
        # Get the python code and GPT code in current row
        python_code = row[4]
        gpt_answer = row[5]

        # Run the model on the sentence, with 100 tokens using version 1.1
        try:
          detectGPT_answer_human_full = model(python_code, 100, "v1.1")
          print('Probability for Human: {}'.format(detectGPT_answer_human_full))
          row.append(detectGPT_answer_human_full)
        except:
          pass
        torch.cuda.empty_cache()
        try:
          detectGPT_answer_GPT_full = model(gpt_answer, 100, "v1.1")
          print('Probability for GPT: {}'.format(detectGPT_answer_GPT_full))
          row.append(detectGPT_answer_GPT_full)
        except:
          pass

        # Append the result to the row and write it to the output file
        writer_obj.writerow(row)
        print('Done -- index {}!\n'.format(row[0]))

        # Replace paths to shutil.copy(path to DetectGPT output file, path to save DetectGPT outputs in mounted GDrive)
        # shutil.copy("/content/DetectGPT/data/output_variant_1.csv", "/content/drive/My Drive/VariantOutputs/") 


